In [1]:
import pandas as pd
from pathlib import Path
import zarr
import dask.array as da
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader
from dask.diagnostics import ProgressBar
ProgressBar().register()

In [2]:
z_path = Path('3e0f709c-c757-4590-901e-081a82843a61-ome.zarr')
# read the image data
store = parse_url(z_path, mode="r").store

In [3]:
reader = Reader(parse_url(z_path))
# nodes may include images, labels etc
nodes = list(reader())

# first node will be the image pixel data
image_node = nodes[0]

dask_data = image_node.data
d0 = dask_data[0]
d0

dask.array<from-zarr, shape=(10240, 11264), dtype=uint16, chunksize=(1024, 1024), chunktype=numpy.ndarray>

In [4]:
from dask_image import ndfilters, ndmeasure, ndmorph
from skimage.morphology import disk

from ome_zarr.writer import write_multiscale
def rm_tree(pth):
    """ Helper function to remove a folder if exists during ome.zarr creation"""
    pth = Path(pth)
    for child in pth.glob('*'):
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()


In [5]:
smoothed = ndfilters.gaussian_filter(d0, sigma=[5, 5])

thresh = ndfilters.threshold_local(smoothed, (d0.chunksize[0]*2, d0.chunksize[1]*2))
th_img = d0 > thresh

bw_img = ndmorph.binary_closing(th_img, structure=disk(15))

label_image, num_labels = ndmeasure.label(bw_img)

In [6]:
p = Path("test_ngff_image-ome.zarr")
if p.exists():
  rm_tree(p)

p.mkdir()

In [7]:
store = parse_url(p, mode='w').store
root = zarr.group(store=store)
tr = image_node.metadata['coordinateTransformations'][0]
ax = image_node.metadata['axes']
# Use OME write multiscale;
write_multiscale([label_image],
        group=root, axes=ax, coordinate_transformations=[tr]
        )

# add omero metadata: the napari ome-zarr plugin uses this to pass rendering
# options to napari.
root.attrs['omero'] = {
        'channels': [{
                'color': 'ffffff',
                'label': 'segmentation',
                'active': True,
                }]
        }

[########################################] | 100% Completed | 114.01 s
